<a href="https://colab.research.google.com/github/sarah-duarte/ciencia_de_dados/blob/main/ciencia_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>